In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

# 定义目录
data_dir = './videos/MRI515_T2'
data_dir = './videos/MRI515_T1'

label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

# 自动分割结果所在目录（使用 nolap 版本）
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')

# 输出 CSV 文件路径（最终长格式结果）
csv_write_path = os.path.join(data_dir, prompt_dir, prompt_dir + '_area_results_long.csv')

# 检查所需目录是否存在
if not os.path.exists(label_dir) or not os.path.exists(auto_seg_dir):
    raise FileNotFoundError("One or more required directories do not exist!")

# 每个像素对应的实际面积（例如，单位为平方毫米）
spacing_factor = 0.6875 * 0.6875  # = 0.47265625

# 定义 4 个类别对应的目标像素值
classes = [50, 100, 150, 200]

results = []

# 处理每个 label 文件
for label_file in os.listdir(label_dir):
    # 假设文件名格式为 "123.png"，提取数字部分作为索引
    try:
        label_index = int(os.path.splitext(label_file)[0])
    except ValueError:
        print(f"Skipping file with non-numeric name: {label_file}")
        continue

    # 构造 label 文件的完整路径
    label_path = os.path.join(label_dir, label_file)
    label_index = str(int(label_index))
    
    # 构造自动分割 mask 的完整路径（4 个类别）
    auto_seg_paths = [
        os.path.join(auto_seg_dir, f"frame_{label_index}_obj_{i}.png")
        for i in range(1, len(classes) + 1)
    ]
    # 检查是否所有自动分割文件均存在
    if not all(os.path.exists(p) for p in auto_seg_paths):
        print(f"Auto segmentation files missing for {label_file}, skipping.")
        continue

    # 读取 label mask
    label_mask = np.array(Image.open(label_path))

    # 读取所有自动分割 mask
    auto_masks = [np.array(Image.open(p)) for p in auto_seg_paths]

    file_result = {"file_name": label_file}

    # 对每个类别进行面积计算
    for idx, class_val in enumerate(classes, start=1):
        # label: threshold处理，选取与目标值相近的像素（例如 ±10）
        label_class_mask = (np.abs(label_mask - class_val) < 10).astype(np.uint8)
        # auto segmentation：对应 obj_i 文件，使用相同的阈值条件
        auto_class_mask = (np.abs(auto_masks[idx - 1] - class_val) < 10).astype(np.uint8)
        # 调整自动分割 mask 的尺寸以匹配 label mask
        target_size = (label_mask.shape[1], label_mask.shape[0])
        auto_class_mask = cv2.resize(auto_class_mask, target_size, interpolation=cv2.INTER_NEAREST)

        # 计算面积：非0像素数 × spacing_factor
        label_area = np.sum(label_class_mask) * spacing_factor
        auto_area  = np.sum(auto_class_mask)  * spacing_factor

        # 保存结果，键名例如 "class1_label_area", "class1_auto_area"
        file_result[f"class{idx}_label_area"] = label_area
        file_result[f"class{idx}_auto_area"] = auto_area

    results.append(file_result)
    # 打印处理结果
    print(f"Processed {label_file}:", end=" ")
    for idx in range(1, len(classes) + 1):
        print(f"Class{idx} -> Label: {file_result[f'class{idx}_label_area']:.2f}, Auto: {file_result[f'class{idx}_auto_area']:.2f}; ", end="")
    print()

# 将宽格式结果转换为长格式
rows = []
for res in results:
    pid = os.path.splitext(res["file_name"])[0]  # 使用文件名（去除扩展名）作为 PID
    for cls in range(1, len(classes) + 1):
        rows.append({
            "PID": pid,
            "Class": cls,
            "Auto Area": res[f"class{cls}_auto_area"],
            "Manual Area": res[f"class{cls}_label_area"]
        })

results_long_df = pd.DataFrame(rows)

# 保存长格式结果到 CSV
results_long_df.to_csv(csv_write_path, index=False)
print(f"Results saved to {csv_write_path}")

# 输出预览
print(results_long_df.head())


Processed 00261.png: Class1 -> Label: 1002.50, Auto: 981.71; Class2 -> Label: 774.68, Auto: 791.23; Class3 -> Label: 700.00, Auto: 656.99; Class4 -> Label: 1192.04, Auto: 1018.57; 
Processed 00212.png: Class1 -> Label: 1504.46, Auto: 1556.93; Class2 -> Label: 1284.68, Auto: 1193.46; Class3 -> Label: 1338.09, Auto: 1129.18; Class4 -> Label: 1762.54, Auto: 1686.44; 
Processed 00378.png: Class1 -> Label: 1347.54, Auto: 1566.86; Class2 -> Label: 643.29, Auto: 385.69; Class3 -> Label: 646.12, Auto: 837.55; Class4 -> Label: 1459.56, Auto: 977.45; 
Processed 00071.png: Class1 -> Label: 1201.49, Auto: 1113.11; Class2 -> Label: 1040.32, Auto: 1049.30; Class3 -> Label: 982.18, Auto: 1115.47; Class4 -> Label: 1360.78, Auto: 1094.20; 
Processed 00328.png: Class1 -> Label: 1677.93, Auto: 2316.02; Class2 -> Label: 1357.94, Auto: 584.68; Class3 -> Label: 1192.98, Auto: 1175.02; Class4 -> Label: 1558.35, Auto: 1290.35; 
Processed 00193.png: Class1 -> Label: 1415.61, Auto: 1806.49; Class2 -> Label: 117

Processed 00398.png: Class1 -> Label: 728.84, Auto: 1023.30; Class2 -> Label: 1057.33, Auto: 617.29; Class3 -> Label: 947.68, Auto: 843.69; Class4 -> Label: 886.23, Auto: 731.20; 
Processed 00301.png: Class1 -> Label: 907.03, Auto: 1027.08; Class2 -> Label: 1116.41, Auto: 1106.96; Class3 -> Label: 834.71, Auto: 851.73; Class4 -> Label: 967.05, Auto: 806.35; 
Processed 00009.png: Class1 -> Label: 1721.41, Auto: 1740.79; Class2 -> Label: 963.27, Auto: 745.38; Class3 -> Label: 975.56, Auto: 782.25; Class4 -> Label: 1985.63, Auto: 1881.17; 
Processed 00102.png: Class1 -> Label: 1380.63, Auto: 2053.69; Class2 -> Label: 1820.20, Auto: 1100.82; Class3 -> Label: 1649.57, Auto: 1094.20; Class4 -> Label: 2141.13, Auto: 1944.98; 
Processed 00329.png: Class1 -> Label: 1866.99, Auto: 2962.61; Class2 -> Label: 1203.38, Auto: 134.23; Class3 -> Label: 1086.16, Auto: 1143.36; Class4 -> Label: 2048.49, Auto: 1753.08; 
Processed 00230.png: Class1 -> Label: 1029.45, Auto: 1451.53; Class2 -> Label: 1177.86

Processed 00059.png: Class1 -> Label: 1241.20, Auto: 1205.27; Class2 -> Label: 929.24, Auto: 907.50; Class3 -> Label: 799.73, Auto: 1885.43; Class4 -> Label: 1226.54, Auto: 615.87; 
Processed 00052.png: Class1 -> Label: 1514.86, Auto: 2255.04; Class2 -> Label: 880.56, Auto: 72.32; Class3 -> Label: 783.19, Auto: 936.33; Class4 -> Label: 1573.47, Auto: 1335.25; 
Processed 00279.png: Class1 -> Label: 1647.68, Auto: 1545.59; Class2 -> Label: 983.12, Auto: 1017.63; Class3 -> Label: 1119.25, Auto: 1026.14; Class4 -> Label: 1663.28, Auto: 1569.69; 
Processed 00265.png: Class1 -> Label: 775.16, Auto: 745.38; Class2 -> Label: 1081.91, Auto: 1118.30; Class3 -> Label: 936.80, Auto: 875.83; Class4 -> Label: 960.44, Auto: 932.08; 
Processed 00225.png: Class1 -> Label: 1443.02, Auto: 997.30; Class2 -> Label: 1291.77, Auto: 1369.76; Class3 -> Label: 1338.56, Auto: 1072.46; Class4 -> Label: 1335.73, Auto: 1217.56; 
Processed 00274.png: Class1 -> Label: 1284.68, Auto: 1161.79; Class2 -> Label: 1051.19,

Processed 00132.png: Class1 -> Label: 1310.68, Auto: 1524.79; Class2 -> Label: 938.22, Auto: 853.62; Class3 -> Label: 979.34, Auto: 920.73; Class4 -> Label: 1376.38, Auto: 1301.70; 
Processed 00390.png: Class1 -> Label: 875.36, Auto: 856.45; Class2 -> Label: 539.77, Auto: 513.78; Class3 -> Label: 584.20, Auto: 521.34; Class4 -> Label: 881.03, Auto: 830.93; 
Processed 00272.png: Class1 -> Label: 1344.23, Auto: 1053.08; Class2 -> Label: 791.70, Auto: 717.96; Class3 -> Label: 957.60, Auto: 824.31; Class4 -> Label: 1374.96, Auto: 1133.90; 
Processed 00412.png: Class1 -> Label: 1519.59, Auto: 2431.82; Class2 -> Label: 886.23, Auto: 510.94; Class3 -> Label: 938.22, Auto: 1577.73; Class4 -> Label: 2021.08, Auto: 1714.80; 
Processed 00190.png: Class1 -> Label: 1453.42, Auto: 1953.02; Class2 -> Label: 1243.09, Auto: 311.48; Class3 -> Label: 1281.37, Auto: 1461.45; Class4 -> Label: 1627.83, Auto: 1001.09; 
Processed 00414.png: Class1 -> Label: 1199.60, Auto: 1260.57; Class2 -> Label: 1150.45, Au

Processed 00179.png: Class1 -> Label: 1638.70, Auto: 1821.62; Class2 -> Label: 922.62, Auto: 782.72; Class3 -> Label: 1006.76, Auto: 979.34; Class4 -> Label: 1736.07, Auto: 1692.58; 
Processed 00375.png: Class1 -> Label: 1130.59, Auto: 1418.44; Class2 -> Label: 950.98, Auto: 661.25; Class3 -> Label: 1005.81, Auto: 925.93; Class4 -> Label: 1423.64, Auto: 1618.85; 
Processed 00312.png: Class1 -> Label: 1215.20, Auto: 1594.27; Class2 -> Label: 1182.11, Auto: 693.39; Class3 -> Label: 1227.02, Auto: 1061.59; Class4 -> Label: 1350.38, Auto: 1278.06; 
Processed 00310.png: Class1 -> Label: 1183.53, Auto: 1203.38; Class2 -> Label: 1014.32, Auto: 782.25; Class3 -> Label: 1049.30, Auto: 952.40; Class4 -> Label: 1545.11, Auto: 1455.78; 
Processed 00385.png: Class1 -> Label: 1346.60, Auto: 1196.77; Class2 -> Label: 959.02, Auto: 940.11; Class3 -> Label: 796.90, Auto: 987.38; Class4 -> Label: 1326.75, Auto: 941.06; 
Processed 00391.png: Class1 -> Label: 1071.51, Auto: 893.32; Class2 -> Label: 807.77

In [2]:
results_long_df

PID  Class    Auto Area  Manual Area
0     00261      1   981.707031  1002.503906
1     00261      2   791.226562   774.683594
2     00261      3   656.992188   700.003906
3     00261      4  1018.574219  1192.039062
4     00212      1  1556.929688  1504.464844
...     ...    ...          ...          ...
1655  00091      4  1106.015625  1104.597656
1656  00377      1  1126.812500  1114.050781
1657  00377      2   975.562500  1017.628906
1658  00377      3  1063.476562  1106.960938
1659  00377      4   817.695312  1164.625000

[1660 rows x 4 columns]

In [3]:
manual_prompt_id = ['00000', '00001', '00002']
# 逻辑：根据 prompt_dir 中是否包含 "1" 或 "2" 来决定去除 manual_prompt_id 中的前几项
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]
results_df = filter_pid(results_long_df)
results_df

Remove prefixes: ['00000']


PID  Class    Auto Area  Manual Area
0     00261      1   981.707031  1002.503906
1     00261      2   791.226562   774.683594
2     00261      3   656.992188   700.003906
3     00261      4  1018.574219  1192.039062
4     00212      1  1556.929688  1504.464844
...     ...    ...          ...          ...
1655  00091      4  1106.015625  1104.597656
1656  00377      1  1126.812500  1114.050781
1657  00377      2   975.562500  1017.628906
1658  00377      3  1063.476562  1106.960938
1659  00377      4   817.695312  1164.625000

[1656 rows x 4 columns]

In [4]:
df_results = results_df.copy()


In [5]:
import numpy as np
from scipy.stats import ks_1samp, norm
from statsmodels.stats.weightstats import ttost_paired
from tqdm import tqdm

# 类别名称映射
class_names = {
    1: "Right_LES",
    2: "Right_MF",
    3: "Left_MF",
    4: "Left_LES"
}
# 自定义打印顺序
print_order = [4, 1, 3, 2]

# 参数
margin_pct = 0.05
n_bootstrap = 10000

# 用于记录每类的最小margin和对应p值
min_margins = []

for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]
    diff = auto - label

    # 正态性检验
    z = (diff - np.mean(diff)) / np.std(diff, ddof=1)
    ks_stat, ks_p = ks_1samp(z, norm.cdf)
    delta = np.mean(label) * margin_pct

    print(f"==== Class: {label_name} ====")
    print(f"Sample size: {min_len}")
    print(f"[KS normality] p = {ks_p:.4f}")

    if ks_p >= 0.05:
        # parametric TOST
        p_tost, *_ = ttost_paired(auto, label, -delta, delta)
        print("→ Using parametric TOST")
        print(f"pTOST = {p_tost:.4f} {'✅ Equivalent' if p_tost < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            p_equiv, *_ = ttost_paired(auto, label, -d, d)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}% at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

    else:
        # bootstrap TOST
        inside = 0
        for _ in tqdm(range(n_bootstrap), leave=False):
            idx = np.random.choice(len(diff), len(diff), replace=True)
            mean_diff = np.mean(diff[idx])
            if -delta < mean_diff < delta:
                inside += 1
        p_boot = 1 - (inside / n_bootstrap)
        print("→ Using bootstrap TOST")
        print(f"bootstrap pTOST = {p_boot:.4f} {'✅ Equivalent' if p_boot < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            inside_test = 0
            for _ in range(10000):
                idx = np.random.choice(len(diff), len(diff), replace=True)
                mean_diff = np.mean(diff[idx])
                if -d < mean_diff < d:
                    inside_test += 1
            p_equiv = 1 - (inside_test / 10000)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}%, at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

# 最后统一打印
print("### Minimum Margin Summary ###")
for margin, pval in min_margins:
    if not np.isnan(margin):
        print(f"{margin:.2f}% ({pval:.3f})")
    else:
        print("N/A (no equivalence within 50%)")


==== Class: Left_LES ====
Sample size: 414
[KS normality] p = 0.0000


→ Using bootstrap TOST
bootstrap pTOST = 1.0000 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±15.40%, at p value 0.045

==== Class: Right_LES ====
Sample size: 414
[KS normality] p = 0.0932
→ Using parametric TOST
pTOST = 0.9996 ❌ Not equivalent
→ Minimum margin for equivalence: δ = ±11.10% at p value 0.049

==== Class: Left_MF ====
Sample size: 414
[KS normality] p = 0.0000


→ Using bootstrap TOST
bootstrap pTOST = 0.2209 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±6.00%, at p value 0.046

==== Class: Right_MF ====
Sample size: 414
[KS normality] p = 0.0001


→ Using bootstrap TOST
bootstrap pTOST = 1.0000 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±24.30%, at p value 0.040

### Minimum Margin Summary ###
15.40% (0.045)
11.10% (0.049)
6.00% (0.046)
24.30% (0.040)


In [6]:
import pingouin as pg

# 初始化
epsilon = 1e-6
mape_results = []
icc_results = []

# 分析每一类
for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]

    # MAPE
    ape = np.abs((auto - label) / (label + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append([label_name, mape * 100, mape_std * 100])

    # ICC
    df_long = df_class.melt(id_vars=["PID"], value_vars=["Manual Area", "Auto Area"],
                            var_name="Method", value_name="Area")
    icc_df = pg.intraclass_corr(data=df_long, targets="PID", raters="Method", ratings="Area")
    icc_row = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
    icc_value = icc_row["ICC"]
    ci_lower, ci_upper = icc_row.get("CI95%") if "CI95%" in icc_row else (np.nan, np.nan)
    icc_results.append([label_name, icc_value, ci_lower, ci_upper])

# 输出 MAPE
print("\n### MAPE Results ###")
for cls, m, s in mape_results:
    print(f"{cls}: {m:.1f} ± {s:.1f}%")

# 输出 ICC
print("\n### ICC Results ###")
for cls, icc, lo, hi in icc_results:
    print(f"{cls}: {icc:.3f}, [{lo:.2f} - {hi:.2f}]")



### MAPE Results ###
Left_LES: 15.8 ± 14.9%
Right_LES: 20.4 ± 18.1%
Left_MF: 15.6 ± 16.6%
Right_MF: 24.6 ± 21.7%

### ICC Results ###
Left_LES: 0.551, [0.23 - 0.72]
Right_LES: 0.604, [0.49 - 0.69]
Left_MF: 0.537, [0.46 - 0.60]
Right_MF: 0.262, [0.01 - 0.46]
